In [1]:
# 일단 트랜스포머 깔아주시고
!pip install transformers
!pip install --upgrade tensorflow
# 패키지 불러와
import tensorflow as tf
import torch
# 귀찮으니깐 bert tokenizer 쓸께유
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
# 데이터셋 학습/평가 나누기
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import time
import datetime
# 런타임 확인
n_devices = torch.cuda.device_count()
print(n_devices)
for i in range(n_devices):
    print(torch.cuda.get_device_name(i))
# 데이터 불러오기 위해 mount 하자
from google.colab import drive
drive.mount('/content/drive')
# 데이터 변환
import csv
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/android_test/ChatbotData.csv', encoding = "utf-8")
data.shape
data_zero = data[data['label'] == 0]
data_1 = data[data['label']==1]
data_zero.tail()
data_2 = data[data['label']==2]
gas = pd.read_excel("/content/drive/MyDrive/android_test/gas.xlsx")
# 가진 데이터는 data_zero, data_2, gas
gas.shape
# data_1을 샘플로 사용하며 labeling을 0으로 바꿈
data_1.loc[data_1['label']==1, 'label'] = 0
data_1.head()
# 데이터 연결하기
gas_data = pd.concat([data_zero, gas], axis = 0)
gas_data = gas_data.reset_index(drop=True)
gas_data.head()
# 랜덤으로 섞기
gas_data_shuffled = gas_data.sample(frac=1).reset_index(drop = True)
gas_data_shuffled.tail()
train = gas_data_shuffled[:4000]
test = gas_data_shuffled[4000:]
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.A]
sentences[:5]
labels = train['label'].values
import pandas as pd
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요')
print(result)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후
MAX_LEN = 128 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# attention mask
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
# 훈련셋, 검증셋 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels,
                                                                                    random_state=2000,
                                                                                    test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=2000,
                                                       test_size=0.1)
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
batch_size = 32
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in test.A]
# 라벨 데이터
labels = test['label'].values
# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)
# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8
                )
# 에폭수
epochs = 6
# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs
# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
#그래디언트 초기화
model.zero_grad()
# 학습
for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # 시작 시간 설정
    t0 = time.time()
    # 로스 초기화
    total_loss = 0
    # 훈련모드로 변경
    model.train()
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)
        # 로스 구함
        loss = outputs[0]
        # 총 로스 계산
        total_loss += loss.item()
        # Backward 수행으로 그래디언트 계산
        loss.backward()
        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()
        # 스케줄러로 학습률 감소
        scheduler.step()
        # 그래디언트 초기화
        model.zero_grad()
    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")
    #시작 시간 설정
    t0 = time.time()
    # 평가모드로 변경
    model.eval()
    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
        # 로스 구함
        logits = outputs[0]
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")

# 모델 학습 완료 후 모델의 가중치를 저장합니다.
model_save_path = "/content/drive/MyDrive/android_test/bert_finetuned_gas.pth"
torch.save(model.state_dict(), model_save_path)
print("Model saved to:", model_save_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 42.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successf

['안', '##녕', '##하', '##세', '##요']
[CLS] 좋은 선택이길 바랄게요. [SEP]
['[CLS]', '좋은', '선', '##택', '##이', '##길', '바', '##랄', '##게', '##요', '.', '[SEP]']
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 6 ========
Training...

  Average training loss: 0.09
  Training epcoh took: 0:01:13

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:03

======== Epoch 2 / 6 ========
Training...

  Average training loss: 0.03
  Training epcoh took: 0:01:13

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:03

======== Epoch 3 / 6 ========
Training...

  Average training loss: 0.01
  Training epcoh took: 0:01:13

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:03

======== Epoch 4 / 6 ========
Training...

  Average training loss: 0.00
  Training epcoh took: 0:01:13

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:03

======== Epoch 5 / 6 ========
Training...

  Average training loss: 0.00
  Training epcoh took: 0:01:13

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:03

======== Epoch 6 / 6 ========
Training...

  Average training loss: 0.00
  Training epcoh took: 0:01:13

Running Validation...
  Accuracy: 

In [2]:
!pip install keras
!pip install tensorflow==2.12  # 이 버전은 대체로 안정적이며 대부분의 패키지와 호환됩니다.
!pip install typing-extensions>=4.6.1
!pip install transformers
!pip install flask-ngrok

from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)  # Flask 앱을 ngrok에 연결

# BERT 모델과 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_model():
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
    model.load_state_dict(torch.load("/content/drive/MyDrive/android_test/bert_finetuned_gas.pth"))  # 미리 학습한 가중치 로드
    model.to(device)
    model.eval()
    return model

def convert_input_data(sentences):
    # 주어진 문장을 BERT 입력 형식으로 변환
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 128
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)
    return inputs, masks

def predict(model, message):
    # 주어진 메시지에 대해 모델 예측 수행
    inputs = tokenizer(message, return_tensors='pt', padding=True, truncation=True)
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    predictions = np.argmax(logits.cpu().numpy(), axis=1)

    if predictions[0] == 1:
        return "가스라이팅 문장입니다."
    else:
        return "일상 대화 문장입니다."

model = load_model()

@app.route('/analyze', methods=['POST'])
def analyze():
    try:
        data = request.json
        message = data.get("message")

        # 메시지 분석
        prediction = predict(model, message)

        result = {
            "result": prediction  # 분석 결과 반환
        }

        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 38.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.0
    Uninstalling tensorboard-2.13.0:
      Successfully uninstalled tensorboard-2.13.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.13.0
    Uninstalling tensorflow-2.13.0:
      Successfully un

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a53d-34-91-56-44.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:32:32] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:32:34] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:32:40] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:32:46] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:32:49] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:33:09] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:33:13] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:33:18] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:33:21] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:33:25] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:33:27] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2023 09:33:59] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:12